## Introduction

There are 54 stores and 33 prodcut families in the data. The time serie starts from 2013-01-01 and finishes in 2017-08-31. However, you know that Kaggle gives us splitted two data as train and test. The dates in the test data are for the 15 days after the last date in the training data. Date range in the test data will be very important to us while we are defining a cross-validation strategy and creating new features.

Our main mission in this competition is, predicting sales for each product family and store combinations.

There are 6 data that we will study on them step by step.

1. Train
2. Test
3. Store
4. Transactions
5. Holidays and Events
6. Daily Oil Price

`The train data` contains time series of the stores and the product families combination. The sales column gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).The onpromotion column gives the total number of items in a product family that were being promoted at a store at a given date.

`Stores data` gives some information about stores such as city, state, type, cluster.

`Transaction data` is highly correlated with train's sales column. You can understand the sales patterns of the stores.

`Holidays and events data` is a meta data. This data is quite valuable to understand past sales, trend and seasonality components. However, it needs to be arranged. You are going to find a comprehensive data manipulation for this data. That part will be one of the most important chapter in this notebook.

`Daily Oil Price data` is another data which will help us. Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices. That's why, it will help us to understand which product families affected in positive or negative way by oil price.

When you look at the data description, you will see "Additional Notes". These notes may be significant to catch some patterns or anomalies. I'm sharing them with you to remember.
* Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
* A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

## Import Libraries

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

## Loading Data

In [82]:
# Import Data
train = pd.read_csv("./store-sales-time-series-forecasting/train.csv", index_col=0)
test = pd.read_csv("./store-sales-time-series-forecasting/test.csv")
oil = pd.read_csv("./store-sales-time-series-forecasting/oil.csv")
holidays_events = pd.read_csv("./store-sales-time-series-forecasting/holidays_events.csv")
sample_submission = pd.read_csv("./store-sales-time-series-forecasting/sample_submission.csv")
stores = pd.read_csv("./store-sales-time-series-forecasting/stores.csv")
transactions = pd.read_csv("./store-sales-time-series-forecasting/transactions.csv").sort_values(['store_nbr', 'date'])

# Datetime
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
oil["date"] = pd.to_datetime(oil.date)
holidays_events["date"] = pd.to_datetime(holidays_events.date)
transactions["date"] = pd.to_datetime(transactions.date)

# Data Types
train['onpromotion'] = train['onpromotion'].astype("float16")
train['sales'] = train['sales'].astype('float32')
stores['cluster'] = stores['cluster'].astype('int8')

In [83]:
train

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.000000,0.0
1,2013-01-01,1,BABY CARE,0.000000,0.0
2,2013-01-01,1,BEAUTY,0.000000,0.0
3,2013-01-01,1,BEVERAGES,0.000000,0.0
4,2013-01-01,1,BOOKS,0.000000,0.0
...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.132996,15.0
3000884,2017-08-15,9,PREPARED FOODS,154.552994,8.0
3000885,2017-08-15,9,PRODUCE,2419.729004,148.0


* `The training data`, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
* `store_nbr` identifies the store at which the products are sold.
* `family` identifies the type of product sold.
* `sales` gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
* `onpromotion` gives the total number of items in a product family that were being promoted at a store at a given date.

In [109]:
test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,20.0
1,3000889,2017-08-16,1,BABY CARE,1.0
2,3000890,2017-08-16,1,BEAUTY,8.0
3,3000891,2017-08-16,1,BEVERAGES,560.0
4,3000892,2017-08-16,1,BOOKS,1.0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,54.0
28508,3029396,2017-08-31,9,PREPARED FOODS,10.0
28509,3029397,2017-08-31,9,PRODUCE,302.0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,15.0


* The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
* The dates in the test data are for the 15 days after the last date in the training data.

In [85]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


* Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

In [86]:
holidays_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


* Holidays and Events, with metadata
* NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
* Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

In [87]:
sample_submission

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


* A sample submission file in the correct format.

In [88]:
display(stores.head())
display(stores.tail())

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


,store_nbr,city,state,type,cluster
49,50,Ambato,Tungurahua,A,14
50,51,Guayaquil,Guayas,A,17
51,52,Manta,Manabi,A,11
52,53,Manta,Manabi,D,13
53,54,El Carmen,Manabi,C,3


* Store metadata, including city, state, type, and cluster.
* cluster is a grouping of similar stores.

In [89]:
transactions

,date,store_nbr,transactions
1,2013-01-02,1,2111
47,2013-01-03,1,1833
93,2013-01-04,1,1863
139,2013-01-05,1,1509
185,2013-01-06,1,520
...,...,...,...
83271,2017-08-11,54,768
83325,2017-08-12,54,903
83379,2017-08-13,54,1054
83433,2017-08-14,54,818


* Transaction data is highly correlated with train's sales column. You can understand the sales patterns of the stores.

## Checking The Types of Data

In [90]:
train.info()
print('\nIs there Null / Missing Values? ', train.isna().any().any())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   store_nbr    int64         
 2   family       object        
 3   sales        float32       
 4   onpromotion  float16       
dtypes: datetime64[ns](1), float16(1), float32(1), int64(1), object(1)
memory usage: 108.8+ MB

Is there Null / Missing Values?  False


* Data has float, integer, datetime, and object values.
* No variable column has null/missing values.

In [91]:
test.info()
print('\nIs there Null / Missing Values? ', test.isna().any().any())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           28512 non-null  int64         
 1   date         28512 non-null  datetime64[ns]
 2   store_nbr    28512 non-null  int64         
 3   family       28512 non-null  object        
 4   onpromotion  28512 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.1+ MB

Is there Null / Missing Values?  False


* Data has float, integer, datetime, and object values.
* No variable column has null/missing values.

In [92]:
holidays_events.info()
print('\nIs there Null / Missing Values? ', holidays_events.isna().any().any())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         350 non-null    datetime64[ns]
 1   type         350 non-null    object        
 2   locale       350 non-null    object        
 3   locale_name  350 non-null    object        
 4   description  350 non-null    object        
 5   transferred  350 non-null    bool          
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 14.1+ KB

Is there Null / Missing Values?  False


* Data has only object and boolean values.
* No variable column has null/missing values.

In [93]:
oil.info()
print('\nIs there Null / Missing Values? ', oil.isna().any().any())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1218 non-null   datetime64[ns]
 1   dcoilwtico  1175 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.2 KB

Is there Null / Missing Values?  True


* Data has only datetime and float values.
* dcoilwtico column has 43 null/missing values.

In [94]:
sample_submission.info()
print('\nIs there Null / Missing Values? ', sample_submission.isna().any().any())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      28512 non-null  int64  
 1   sales   28512 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 445.6 KB

Is there Null / Missing Values?  False


* Data has only integer and float values.
* No variable column has null/missing values.

In [95]:
stores.info()
print('\nIs there Null / Missing Values? ', stores.isna().any().any())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int8  
dtypes: int64(1), int8(1), object(3)
memory usage: 1.9+ KB

Is there Null / Missing Values?  False


* Data has only integer and object values.
* No variable column has null/missing values.

In [96]:
transactions.info()
print('\nIs there Null / Missing Values? ', transactions.isna().any().any())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83488 entries, 1 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 2.5 MB

Is there Null / Missing Values?  False


* Data has only datetime and integer values.
* No variable column has null/missing values.

In [98]:
train.shape

(3000888, 5)

* Dataset comprises of 3.000.888 observations and 5 characteristics.
* Out of which one is dependent variable (sales) and rest 4 are independent variables.

In [113]:
test.drop(columns=['id']).shape

(28512, 4)

* Dataset comprises of 28.512 observations and 4 characteristics.

In [100]:
oil.shape

(1218, 2)

* Dataset comprises of 1218 observations and 2 characteristics.

In [101]:
holidays_events.shape

(350, 6)

* Dataset comprises of 350 observations and 6 characteristics.

In [102]:
sample_submission.shape

(28512, 2)

* Dataset comprises of 28.512 observations and 2 characteristics.

In [103]:
stores.shape

(54, 5)

* Dataset comprises of 54 observations and 5 characteristics.

In [105]:
transactions.shape

(83488, 3)

* Dataset comprises of 83.488 observations and 3 characteristics.